### Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as st
import pickle
import os

from pprint import pprint
from copy import deepcopy
from collections import Counter
from collections.abc import Iterable
import holidays

# Plotting
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno

# Time series analysis
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.seasonal import STL

# Feature Selectionb
from boruta import BorutaPy
import tsfresh as tsf

# Sklearn
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.pipeline import Pipeline

# Non-deep learning ML models
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor

# Hyper-parameter tunning
from hyperopt import fmin, hp, tpe, STATUS_OK
from hyperopt.fmin import generate_trials_to_calculate
from hyperopt.pyll.base import scope

import warnings
warnings.filterwarnings("ignore")

In [2]:
import logging

logger = logging.getLogger(__name__)

In [3]:
pd.options.display.max_rows = 100
pd.set_option('display.max_columns', None)

# Explanatory Data Analysis

### Load Datasets
- The **energy file** records Spain’s hourly electricity prices, in Euros per mega watt hour (€/MWh); electricity generation by type of origin (coal, gas, wind power, etc.) in MWh; and energy demand (“load”) in MWh.
- The **weather file** offers the hourly records of five major Spanish cities.
- We will aim to forecast **one lag ahead**

In [4]:
# Energy Dataset
energy_path = os.path.join("datasets", "energy_dataset.csv")
energy_df = pd.read_csv(energy_path, header=0, parse_dates=["time"])

# Weather Features
weather_path = os.path.join("datasets", "weather_features.csv")
weather_df = pd.read_csv(weather_path, header=0, parse_dates=["dt_iso"])

FileNotFoundError: [Errno 2] No such file or directory: 'datasets/energy_dataset.csv'

### Energy Data - Data Analysis & Data Cleaning

In [5]:
# Prepare idx
energy_df['time'] = pd.to_datetime(energy_df['time'], utc=True, infer_datetime_format=True).dt.tz_localize(None)
energy_df.set_index('time', inplace=True)

#### Basic Characteristics

In [ ]:
energy_df.head()

In [ ]:
energy_df.describe()

In [ ]:
energy_df.info()

#### Missing Values
- We observe two empty feature columns. On the right-hand side, we notice a few gaps in individual rows of the other columns.

In [ ]:
msno.matrix(energy_df)

In [10]:
# Drop columns with all null values
energy_df.dropna(axis=1, how="all", inplace=True)

In [11]:
# Interpolate missing values from remaining rows
energy_df = energy_df.interpolate(method ="bfill")

In [12]:
# Drop columns with all values equal to 0
energy_df = energy_df.loc[:, (energy_df!=0).any(axis=0)]

In [13]:
# Drop "forecast" columns
energy_df = energy_df.drop(energy_df.filter(regex="forecast").columns, axis=1, errors="ignore")

#### Rename columns
- The original column names contain some spaces and special characters.

In [14]:
rename_dict = {
    'generation biomass': 'gen_bio', 
    'generation fossil brown coal/lignite': 'gen_lig', 
    'generation fossil coal-derived gas': 'gen_coal_gas', 
    'generation fossil gas': 'gen_gas', 
    'generation fossil hard coal': 'gen_coal', 
    'generation fossil oil': 'gen_oil', 
    'generation fossil oil shale': 'gen_oil_shale', 
    'generation fossil peat': 'gen_peat', 
    'generation geothermal': 'gen_geo', 
    'generation hydro pumped storage consumption': 'gen_hyd_pump', 
    'generation hydro run-of-river and poundage': 'gen_hyd_river', 
    'generation hydro water reservoir': 'gen_hyd_res', 
    'generation marine': 'gen_mar', 
    'generation nuclear': 'gen_nuc', 
    'generation other': 'gen_other', 
    'generation other renewable': 'gen_oth_renew', 
    'generation solar': 'gen_sol', 
    'generation waste': 'gen_waste', 
    'generation wind offshore': 'gen_wind_off', 
    'generation wind onshore': 'gen_wind_on', 
    'total load actual': 'load_actual', 
    'price day ahead': 'price_dayahead', 
    'price actual': 'price'
}

In [15]:
energy_df.rename(columns=rename_dict, inplace=True)

#### Duplicates

In [ ]:
# Show duplicated idxs
energy_df.loc[energy_df.index.duplicated()]

In [17]:
# Remove duplicated rows
energy_df = energy_df.loc[~energy_df.index.duplicated()]

In [18]:
# Remove duplicated columns
energy_df = energy_df.loc[:, ~energy_df.columns.duplicated(keep='first')]

#### Extreme values

In [19]:
# Calculate the absolute z_scores
z_scores = np.abs(st.zscore(energy_df))

In [ ]:
z_scores.head()

In [21]:
# Remove values where z_score is over 2.5 stdev
for col in energy_df.columns:
    energy_df.loc[z_scores[col] > 2.5] = np.nan

In [22]:
# Replace outliers with mean/median values for that column
means_dict = {col: energy_df[col].median() for col in energy_df.columns}

In [23]:
energy_df.fillna(value=means_dict, inplace=True)

In [ ]:
energy_df.describe()

### Target Anlysis
- A lineplot of the hourly prices shows that the curve that does not appear to follow a stable long-term trend or a single seasonality pattern, but fluctuates around a mean price of €60. 
- Since we deal with hourly prices, there might also be seasonalities related to **hours**, **weekdays** & **months**

In [ ]:
plt.figure(100, figsize=(20, 7))
sns.lineplot(
    x="time", 
    y="price", 
    data=energy_df, 
    palette="coolwarm"
)

#### Auto-Correlations & Partial Auto-Correlations
- Both plots show statistically significant AC values and PAC values
- This indicates that the data is autocorrelated, which could imply that there are some seasonality effects.

In [ ]:
plt.figure(100, figsize=(20, 7))
plot_acf(energy_df['price'], lags=20, alpha=0.05)
plt.show()

In [ ]:
plot_pacf(energy_df['price'], lags=20, alpha=0.05)
plt.show()

#### Seasonal decomposition
- Decomposing the time series with both **weekly** and **monthly** frequencies portray a decisive seasonality component for the time series.

In [28]:
# Define data to plot
last_year = energy_df.loc[energy_df.index > '2018', 'price']

In [ ]:
# Plot Weekly frequency
decomp = seasonal_decompose(last_year, period=24*7)

decomp.plot()
plt.show()

In [ ]:
# Plot Monthly frequency
decomp = seasonal_decompose(last_year, period=24*30)

decomp.plot()
plt.show()

### Weather Data - Data Analysis & Data Cleaning

In [31]:
# Prepare idx
weather_df['dt_iso'] = pd.to_datetime(weather_df['dt_iso'], utc=True, infer_datetime_format=True).dt.tz_localize(None)
weather_df.set_index('dt_iso', inplace=True)

#### Basic Characteristics

In [ ]:
weather_df.head()

In [ ]:
weather_df.city_name.unique()

In [ ]:
weather_df.describe()

In [ ]:
weather_df.info()

#### Missing Values
- No missing values were found

In [ ]:
msno.matrix(weather_df)

In [37]:
# Drop columns with all values equal to 0
weather_df = weather_df.loc[:, (weather_df!=0).any(axis=0)]

#### Manage Columns

In [38]:
# drop unnecessary columns
drop_cols = ["rain_3h", "weather_id", "weather_main", "weather_description", "weather_icon"]
weather_df.drop(drop_cols, inplace=True, axis=1, errors="ignore")

#### Basic Transformations

In [39]:
# temperature: kelvin to celsius
temp_cols = [col for col in weather_df.columns if "temp" in col]
weather_df[temp_cols] = weather_df[temp_cols].filter(like="temp").applymap(lambda t: t - 273.15)

In [40]:
# convert int and float64 columns to float32
intcols = list(weather_df.dtypes[weather_df.dtypes == np.int64].index)
weather_df[intcols] = weather_df[intcols].applymap(np.float32)

f64cols = list(weather_df.dtypes[weather_df.dtypes == np.float64].index)
weather_df[f64cols] = weather_df[f64cols].applymap(np.float32)

f32cols = list(weather_df.dtypes[weather_df.dtypes == np.float32].index)

In [ ]:
weather_df.info()

#### Duplicates

In [42]:
# Remove duplicated rows
weather_df = (
    weather_df
    .reset_index()
    .drop_duplicates(subset=["dt_iso", "city_name"], keep="first")
    .set_index('dt_iso')
)

In [43]:
# Remove duplicated columns
weather_df = weather_df.loc[:, ~weather_df.columns.duplicated(keep='first')]

#### Re-group DataFrames

In [44]:
gb_df = weather_df.groupby("city_name")

In [45]:
def format_df(df: pd.DataFrame, city: str):
    df.drop(columns=['city_name'], inplace=True)
    return df.add_suffix(f"_{city.replace(' ', '')}")

concat_weather_df = pd.concat(
    [format_df(gb_df.get_group(city), city) for city in gb_df.groups.keys()],
    axis=1
).interpolate()

In [ ]:
concat_weather_df.head()

#### Extreme values

In [47]:
# Calculate the absolute z_scores
num_cols = list(concat_weather_df.select_dtypes(include=['number']).columns)
z_scores = np.abs(st.zscore(concat_weather_df[num_cols]))

In [ ]:
z_scores.head()

In [49]:
# Remove values where z_score is over 2.5 stdev
for col in num_cols:
    concat_weather_df.loc[z_scores[col] > 2.5] = np.nan

In [50]:
# Replace outliers with mean values for that column
means_dict = {col: concat_weather_df[col].mean() for col in num_cols}

In [51]:
concat_weather_df.fillna(value=means_dict, inplace=True)

In [ ]:
concat_weather_df.describe()

### Merge DataFrames

In [53]:
# Energy & Weather Concat
df = pd.concat([energy_df, concat_weather_df], axis=1)

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.describe()

Define ***Target***

In [57]:
y = df['price']

Define ***Features***
- Features will be *lagged* one observation, as we will be forecasting *one period ahead*

In [58]:
X_forecast = pd.DataFrame(
    index=df.index[-1:] + pd.Timedelta(minutes=60),
    columns=df.columns.tolist()
)

In [59]:
X = (
    pd.concat([df, X_forecast])
    .shift(1)
    .fillna(method='bfill')
    .rename(columns={'price': 'lagged_price'})
)

In [ ]:
X.shape

In [ ]:
pd.concat([y, X[['lagged_price']]], axis=1)

# Feature Engineering

### Features Analysis

In [62]:
# Feature Correlations
df_corr = pd.concat([y, X.loc[X.index.isin(y.index)]], axis=1).corr(method="pearson")

In [63]:
# Find threshold for features filtering
thresh = np.max([np.quantile(np.abs(df_corr['price'].dropna()), 0.25), 0.1])

In [ ]:
thresh

In [65]:
# Pick top features and filter df_corr
top_candidates = list(df_corr.loc[np.abs(df_corr['price']).sort_values() > thresh].index)

In [ ]:
len(top_candidates)

In [67]:
df_corr = (
    df_corr
    .loc[top_candidates, top_candidates]
    .sort_values(by=['price'], ascending=False)
    .round(3)
)

df_corr = df_corr[df_corr.index.tolist()]

In [ ]:
plt.figure(figsize = (17,17))
sns.set(font_scale=0.75)
ax = sns.heatmap(
    df_corr, 
    annot=True, 
    square=True, 
    linewidths=.75, 
    cmap="coolwarm", 
    fmt = ".2f", 
    annot_kws = {"size": 11}
)
ax.xaxis.tick_bottom()
plt.title("correlation matrix")
plt.show()

In [ ]:
# Plot features distributions
X = X.filter(top_candidates)

rows = int(np.ceil(X.shape[1] / 4))
f, ax = plt.subplots(rows, 4, figsize=(20, rows*4.5), gridspec_kw={'wspace':0.5,'hspace':0.3})

ax = ax.ravel()

for i, col in enumerate(X):
    sns.histplot(X[col].astype(float), ax=ax[i], kde=False)
    ax[i].axvline(x=X[col].mean(), color='k', label='mean')
    ax[i].axvline(x=X[col].median(), color='r', label='median')
    
ax[0].legend();
ax[-2].axis('off')
ax[-1].axis('off');

### Feature Enrichment
- There are multiple strategies for enriching features to boost the achievable forecast accuracy of a model.
- This might include techniques like binning, linear combinations of some key features, laggin features, calculating moving average, exponential moing averages, etc.

In [70]:
def find_optimal_seasonal_period(time_series, max_period=None):
    if max_period is None:
        max_period = len(time_series) // 2

    # Perform seasonal decomposition using STL
    decomposition = seasonal_decompose(time_series, period=max_period)

    # Get the seasonal component
    seasonal_component = decomposition.seasonal.dropna()

    # Compute the periodogram
    n = len(seasonal_component)
    fft_values = np.abs(np.fft.fft(seasonal_component)) ** 2
    fft_values = fft_values[:n // 2]
    frequencies = np.fft.fftfreq(n, 1)
    frequencies = frequencies[:n // 2]

    # Find the index of the maximum frequency
    max_index = np.argmax(fft_values)
    # pprint(fft_values)

    # Calculate the optimal seasonal period
    optimal_period = int(1 / frequencies[max_index])

    # Plot the periodogram (optional)
    plt.plot(1 / frequencies, fft_values)
    plt.xlabel('Seasonal Period')
    plt.ylabel('Periodogram')
    plt.title('Periodogram of Seasonal Component')
    plt.show()

    return optimal_period

In [ ]:
sp = find_optimal_seasonal_period(y.copy(), 24*3*30)

In [ ]:
sp

In [73]:
lag_periods = list(range(1, sp//2 + 1)) + [sp * i for i in range(1, sp//2 + 1)] + [24*31, 24*365]
rolling_windows = [sp//2, sp, 2*sp]

In [ ]:
lag_periods

In [ ]:
rolling_windows

In [76]:
# Lagg Features
def lag_df(df_: pd.DataFrame, lag: int):
    df_[df_.columns] = df_[df_.columns].shift(lag, axis=0)
    
    return (
        df_
        .rename(columns=lambda x: f"{x}_lag_{lag}" if x in df_.columns else x)
        .fillna(method='bfill')
    )


# Simple rolling features
def rolling_df(df_: pd.DataFrame, window: int, agg_fun: str = 'mean'):
    if agg_fun == 'mean':
        df_[df_.columns] = df_.rolling(window=window).mean()
    elif agg_fun == 'std':
        df_[df_.columns] = df_.rolling(window=window).std()
    elif agg_fun == 'max':
        df_[df_.columns] = df_.rolling(window=window).max()
    elif agg_fun == 'min':
        df_[df_.columns] = df_.rolling(window=window).min()
    elif agg_fun == 'min_max':
        df_[df_.columns] = df_.rolling(window=window).max() - df_.rolling(window=window).min()
        
    return (
        df_
        .rename(columns=lambda x: f"{x}_sm_{agg_fun}_{window}" if x in df_.columns else x)
        .fillna(method='bfill')
    )


# Exponential Moving Average
def ema(df_: pd.DataFrame, window: int):
    # ewm(span=window, adjust=False)
    df_[df_.columns] = df_.ewm(span=window, adjust=False).mean()
    
    return (
        df_
        .rename(columns=lambda x: f"{x}_ema_{window}" if x in df_.columns else x)
        .fillna(method='bfill')
    )


# Temporal Embedding Features
def tef(df_: pd.DataFrame, dow: bool = True, dom: bool = True, hod: bool = True):
    # Day of Week
    if dow:
        df_['dow_sin'] = np.sin(2 * np.pi * df_.index.dayofweek / 7)
        df_['day_cos'] = np.cos(2 * np.pi * df_.index.dayofweek / 7)
    
    # Day of Month
    if dom:
        df_['dom_sin'] = np.sin(2 * np.pi * df_.index.day / 31)
        df_['dam_cos'] = np.cos(2 * np.pi * df_.index.day / 31)
        
    # Hour of Day
    if hod:
        df_['hod_sin'] = np.sin(2 * np.pi * df_.index.hour / 24)
        df_['hod_cos'] = np.cos(2 * np.pi * df_.index.hour / 24)
        
    return pd.concat([
        df_.filter(like='sin', axis=1), 
        df_.filter(like='cos', axis=1)
    ], axis=1)


# Time Based Features
def tbf(df_: pd.DataFrame):
    # Time-based Features
    df_['month'] = df_.index.month
    df_['day'] = df_.index.day
    df_['day_of_week'] = df_.index.dayofweek
    df_['hour'] = df_.index.hour
    
    return df_[['month', 'day', 'day_of_week', 'hour']]


# Holiday-based features
def extract_holidays(df_: pd.DataFrame):
    spain_holidays = holidays.CountryHoliday('ES', observed=True)
    df_['is_holiday'] = df_.index.to_series().apply(lambda x: x.date() in spain_holidays)
    
    return df_[['is_holiday']]


# Seasonal Decomposition
def extract_stl(target: pd.DataFrame, seasonal_period: int):
    if seasonal_period % 2 == 0:
        seasonal_period += 1
        
    stl = STL(target.values, period=seasonal_period)
    result = stl.fit()
    
    stl_df = pd.concat(
        [pd.Series(result.trend), pd.Series(result.seasonal), pd.Series(result.resid)], axis=1
    ).rename(columns={
        0: 'stl_trend',
        1: 'stl_season',
        2: 'stl_resid'
    })
    stl_df.index = target.index
    
    return stl_df

In [77]:
lag_df = pd.concat(
    [lag_df(X.copy(), lag=lag) for lag in lag_periods],
    axis=1
)

In [ ]:
lag_df.tail()

In [79]:
sma_df = pd.concat(
    [rolling_df(X.copy(), window=window, agg_fun='mean') for window in rolling_windows],
    axis=1
)

In [ ]:
sma_df.tail()

In [81]:
sm_std_df = pd.concat(
    [rolling_df(X.copy(), window=window, agg_fun='std') for window in rolling_windows],
    axis=1
)

In [ ]:
sm_std_df.tail()

In [83]:
sm_max_df = pd.concat(
    [rolling_df(X.copy(), window=window, agg_fun='max') for window in rolling_windows],
    axis=1
)

In [ ]:
sm_max_df.tail()

In [85]:
sm_min_df = pd.concat(
    [rolling_df(X.copy(), window=window, agg_fun='min') for window in rolling_windows],
    axis=1
)

In [ ]:
sm_min_df.tail()

In [87]:
sm_min_max_df = pd.concat(
    [rolling_df(X.copy(), window=window, agg_fun='min_max') for window in rolling_windows],
    axis=1
)

In [ ]:
sm_min_max_df.tail()

In [89]:
ema_df = pd.concat(
    [ema(X.copy(), window=window) for window in rolling_windows],
    axis=1
)

In [ ]:
ema_df.tail()

In [91]:
tef_df = tef(X.copy())

In [ ]:
tef_df.tail()

In [93]:
tbf_df = tbf(X.copy())

In [ ]:
tbf_df.tail()

In [95]:
holiday_df = extract_holidays(X.copy())

In [ ]:
holiday_df.tail()

In [97]:
stl_df = extract_stl(target=X['lagged_price'], seasonal_period=sp)

In [ ]:
stl_df.tail()

In [99]:
# Concatenate extracted DataFrames
X = pd.concat([
    X, lag_df, sma_df, sm_std_df, sm_max_df, sm_min_df, 
    sm_min_max_df, ema_df, tef_df, tbf_df, holiday_df, stl_df
], axis=1)

In [ ]:
X.tail()

In [ ]:
X.shape

In [ ]:
X.isna().sum().sum()

### Feature Standardizing
- Since all features are numerical, OneHotEncoding will not be needed
- We will use the StandardScaler to transform the numerical featuers

In [103]:
scaler = StandardScaler(with_mean=True, with_std=True)

X_stand = pd.DataFrame(
    scaler.fit_transform(X), 
    columns=X.columns,
    index=X.index
)

In [ ]:
X_stand.head()

### Feature Selection
- Similarly to the feature enrichment section, there are multiple ways to select the best subset of features that will aid our model, without overfitting.
- We will start by filtering features based on their correlation with the **target**
- Then, we will then filter out **colinear** features
- Finally, we will leverage the **sklearn.feature_selection**, **boruta** and **tsfresh** libraries to select the best features.

In [105]:
# Target-Feature Correlation Based Filtering
def target_feature_correl_filter(
    y: pd.Series,
    X: pd.DataFrame,
    q_thresh: float = 0.3,
    debug: bool = False
):
    # Prepare DataFrames
    intersection = y.index.intersection(X.index)
    num_cols = list(X.select_dtypes(include=['number']).columns)

    y = y.loc[intersection]
    X = X.loc[intersection, num_cols]

    # Calculate Correlations with target
    tf_corr_df = pd.DataFrame(columns=[y.name])
    for c in X.columns:
        tf_corr_df.loc[c] = [abs(y.corr(X[c]))]
    
    tf_corr_df = tf_corr_df.sort_values(by=[y.name], ascending=False)
    
    if debug:
        print(f'{tf_corr_df.head()}\n')

    # Define threshold
    threshold = np.quantile(tf_corr_df[y.name].dropna(), q_thresh)
    
    return tf_corr_df.loc[tf_corr_df[y.name] > threshold].index.tolist()

In [ ]:
X_stand.shape

In [107]:
correl_filter = target_feature_correl_filter(
    y=y.copy(),
    X=X_stand.copy(),
    q_thresh=0.3,
    debug=False
)

In [108]:
X_stand = X_stand.loc[:, correl_filter]

In [ ]:
X_stand.shape

In [110]:
def colinear_feature_filter(
    X: pd.DataFrame,
    thresh: float = 0.9
):
    cm = pd.DataFrame(X.corr().applymap(lambda x: 100 * np.abs(x))).fillna(100)
    filtered_features = cm.columns.tolist()
    
    i = 0
    while i < len(filtered_features):
        keep_feature = filtered_features[i]
        skip_features = cm.loc[
            (cm[keep_feature] < 100) &
            (cm[keep_feature] >= thresh*100)
        ][keep_feature].index.tolist()
        
        if len(skip_features) > 0:
            filtered_features = [c for c in filtered_features if c not in skip_features]
        i += 1
    
    return filtered_features

In [111]:
colinear_filter = colinear_feature_filter(
    X_stand.copy(), 
    thresh=0.9
)

In [112]:
X_stand = X_stand.loc[:, colinear_filter]

In [ ]:
X_stand.shape

In [114]:
def select_k_best_features(
    y: pd.Series,
    X: pd.DataFrame,
    perc: float = 0.1
):
    # Prepare DataFrames
    intersection = y.index.intersection(X.index)
    
    binary_features = [col for col in X.columns if X[col].nunique() == 2]
    non_binary_features = [col for col in X.columns if X[col].nunique() > 2]

    y = y.loc[intersection]
    X_binary = X.loc[intersection, binary_features]
    X_non_binary = X.loc[intersection, non_binary_features]
    
    # Prepare selected_featuers
    selected_featuers = []
    
    # Select Binary Features
    if X_binary.shape[1] > 0:
        k = int(perc * len(binary_features))
        
        selector = SelectKBest(score_func=mutual_info_regression, k=k)
        selector.fit(X_binary, y)

        selected_featuers.extend(X_binary.columns[selector.get_support()].tolist())
    
    # Select Non-Binary Features
    if X_non_binary.shape[1] > 0:
        k = int(perc * len(non_binary_features))
        
        selector = SelectKBest(score_func=f_regression, k=k)
        selector.fit(X_non_binary, y)

        selected_featuers.extend(X_non_binary.columns[selector.get_support()].tolist())
        
    return selected_featuers

In [115]:
k_best_features = select_k_best_features(
    y.copy(),
    X_stand.copy(),
    perc=0.2
)

In [ ]:
len(k_best_features)

In [117]:
def select_boruta_features(
    y: pd.Series,
    X: pd.DataFrame,
):
    # Prepare DataFrames
    intersection = y.index.intersection(X.index)

    y = y.loc[intersection]
    X = X.loc[intersection]
    
    # Choose the model
    model = LGBMRegressor(num_iterations=100, random_state=23111997)

    # Instanciate and fit the BorutaPy estimator
    boruta = BorutaPy(model, n_estimators='auto', verbose=0, random_state=23111997)            
    boruta.fit(
        np.array(X), 
        np.array(y)
    )

    # Extract features
    return X.columns[boruta.support_].tolist()

In [118]:
boruta_features = select_boruta_features(
    y.copy(),
    X_stand.copy()
)

In [ ]:
len(boruta_features)

In [120]:
def select_tsfresh_features(
    y: pd.Series,
    X: pd.DataFrame,
    p_value: float = 0.01,
    top_k: int = None
):
    # Prepare DataFrames
    intersection = y.index.intersection(X.index)

    y = y.loc[intersection]
    X = X.loc[intersection]
    
    # Run TSFresh Feature Selection
    relevance_table = tsf.feature_selection.relevance.calculate_relevance_table(X, y)
    relevance_table = relevance_table[relevance_table.relevant].sort_values("p_value")
    relevance_table = relevance_table.loc[relevance_table['p_value'] < p_value]
    
    selected_featuers = list(relevance_table["feature"].values)
    
    # Extract Features
    if top_k is not None:
        return selected_featuers[:top_k]
    return selected_featuers

In [121]:
tsfresh_features = select_tsfresh_features(
    y.copy(),
    X_stand.copy(),
    p_value=0.01,
    top_k=50
)

In [ ]:
len(tsfresh_features)

In [123]:
# Merge Features
selected_features = list(set(k_best_features + boruta_features + tsfresh_features))

In [ ]:
len(selected_features)

In [125]:
X_stand = X_stand.loc[:, selected_features]

In [ ]:
X_stand.shape

### Selected Feature Analysis

In [127]:
df_corr = pd.concat([y, X_stand.loc[X_stand.index.isin(y.index)]], axis=1).corr(method="pearson")

In [128]:
df_corr = (
    df_corr
    .sort_values(by=['price'], ascending=False)
    .round(3)
)

df_corr = df_corr[df_corr.index.tolist()]

In [ ]:
plt.figure(figsize = (17,17))
sns.set(font_scale=0.75)
ax = sns.heatmap(
    df_corr, 
    annot=True, 
    square=True, 
    linewidths=.75, cmap="coolwarm", 
    fmt = ".2f", 
    annot_kws = {"size": 11}
)
ax.xaxis.tick_bottom()
plt.title("correlation matrix")
plt.show()

# Modeling

In [131]:
class Model:
    save_attrs = [
        'algorithm',
        'hyper_parameters',
        'fitted'
    ]
    
    def __init__(
        self,
        algorithm: str = None,
        hyper_parameters: dict = None,
    ) -> None:
        self.model = None
        self.algorithm = algorithm
        self.hyper_parameters = None
        if hyper_parameters is not None:
            self.hyper_parameters = self.prepare_hyper_parameters(deepcopy(hyper_parameters))
            
        self.fitted = False
        
    def prepare_hyper_parameters(
        self,
        hyper_parameters: dict
    ):
        if self.algorithm == 'sarimax':
            # Order
            if 'order' not in hyper_parameters:
                order = (
                    hyper_parameters['p'], 
                    hyper_parameters['d'], 
                    hyper_parameters['q']
                )
            else:
                order = hyper_parameters['order']
            
            # Seasonal Order
            if 'seasonal_order' not in hyper_parameters:                
                seasonal_order = (
                    hyper_parameters['seasonal_P'], 
                    hyper_parameters['seasonal_D'],
                    hyper_parameters['seasonal_Q'], 
                    hyper_parameters['seasonal_S']
                )
            else:
                seasonal_order = hyper_parameters['seasonal_order']
            
            # Trend
            if 'trend' not in hyper_parameters:
                trend = hyper_parameters['sarimax.trend']
            else:
                trend = hyper_parameters['trend']
            
            hyper_parameters = {
                'order': order,  # (p, d, q)
                'seasonal_order': seasonal_order,  # (P, D, Q, S)
                'trend': trend, 
                'measurement_error': False,
                'time_varying_regression': False, 
                'mle_regression': True,
                'simple_differencing': False,
                'enforce_stationarity': False, 
                'enforce_invertibility': False,
                'hamilton_representation': False, 
                'concentrate_scale': False,
                'trend_offset': 1, 
                'use_exact_diffuse': False, 
                'dates': None
            }
            
        if self.algorithm == 'random_forest':
            names = list(hyper_parameters.keys()).copy()
            for param_name in names:
                if 'random_forest.' in param_name:
                    correct_name = param_name.replace('random_forest.', '')
                    hyper_parameters[correct_name] = hyper_parameters.pop(param_name)
            
            hyper_parameters.update(**{
                'n_jobs': -1,
                'random_state': 23111997
            })
        
        if self.algorithm == 'lightgbm':
            names = list(hyper_parameters.keys()).copy()
            for param_name in names:
                if 'lightgbm.' in param_name:
                    correct_name = param_name.replace('lightgbm.', '')
                    hyper_parameters[correct_name] = hyper_parameters.pop(param_name)
                
            hyper_parameters.update(**{
                "objective": 'regression',
                "importance_type": 'split',
                "random_state": 23111997,
                "n_jobs": -1
            })
                
        return hyper_parameters
                
    def build(
        self,
        train_target: pd.Series = None, 
        train_features: pd.DataFrame = None
    ):
        if self.algorithm == 'expo_smooth':
            self.model = ExponentialSmoothing(
                train_target,
                trend=self.hyper_parameters['expo_smooth.trend'], 
                damped_trend=self.hyper_parameters['damped_trend'], 
                seasonal=self.hyper_parameters['seasonal'], 
                seasonal_periods=self.hyper_parameters['seasonal_periods']
            )
            
        elif self.algorithm == 'sarimax':
            if self.model is not None:
                sarimax_parameters['start_ar_lags'] = self.model.ar_lags
                sarimax_parameters['start_ma_lags'] = self.model.ma_lags
            
            self.model = SARIMAX(
                endog=train_target.values.astype(float),
                exog=train_features.values.astype(float),
                **sarimax_parameters
            )
            
        elif self.algorithm == 'random_forest':
            self.model = RandomForestRegressor(**self.hyper_parameters)
            
        elif self.algorithm == 'lightgbm':
            self.model = LGBMRegressor(**self.hyper_parameters)
        
        self.fitted = False
            
    def fit(
        self,
        train_target: pd.Series = None, 
        train_features: pd.DataFrame = None
    ):
        if self.algorithm == 'expo_smooth':
            self.model.fit()
            
        elif self.algorithm == 'sarimax':
            self.model = self.model.fit(
                disp=False, 
                maxiter=50
            )
        
        elif self.algorithm == 'random_forest':
            self.model.fit(
                train_features.values.astype(float), 
                train_target.values.astype(float)
            )
        
        elif self.algorithm == 'lightgbm':
            self.model.fit(
                train_features.values.astype(float), 
                train_target.values.astype(float)
            )
            
        self.fitted = True
            
    def predict(
        self,
        train_target: pd.Series = None,
        forecast_features: pd.DataFrame = None, 
        forecast_dates: Iterable = None
    ):
        if self.algorithm == 'expo_smooth':
            return self.model.predict(
                self.model.params,
                start=forecast_dates[0],
                end=forecast_dates[-1],
            )
        
        elif self.algorithm == 'sarimax':
            return self.model.forecast(
                steps=forecast_features.shape[0],
                exog=forecast_features.values.astype(float)
            )
        
        elif self.algorithm == 'random_forest':
            return self.model.predict(
                forecast_features.values.astype(float)
            )
        
        elif self.algorithm == 'lightgbm':
            return self.model.predict(
                forecast_features.values.astype(float)
            )
        
    def save(
        self,
        as_champion: bool = True
    ):
        save_attrs = {
            attr_name: attr_value for attr_name, attr_value in self.__dict__.items()
            if attr_name in self.save_attrs
        }
        
        if as_champion:
            print('Saving new champion.\n')
            
            # Define paths
            model_path = os.path.join('models', 'champion', 'champion.pickle')
            attrs_path = os.path.join('models', 'champion', 'attrs.pickle')
            
            
        else:
            print('Saving new challenger.\n')
            
            # Define paths
            model_path = os.path.join('models', 'challenger', 'challenger.pickle')
            attrs_path = os.path.join('models', 'challenger', 'attrs.pickle')
                
        # Save self.model            
        with open(model_path, 'wb') as file:
            pickle.dump(self.model, file)

        # Save attrs            
        with open(attrs_path, 'wb') as file:
            pickle.dump(save_attrs, file)
            
    def load(
        self,
        champion: bool = True
    ):
        if champion:
            print('Loading champion.\n')
            
            # Define paths
            model_path = os.path.join('models', 'champion', 'champion.pickle')
            attrs_path = os.path.join('models', 'champion', 'attrs.pickle')
            
            
        else:
            print('Loading challenger.\n')
            
            # Define paths
            model_path = os.path.join('models', 'challenger', 'challenger.pickle')
            attrs_path = os.path.join('models', 'challenger', 'attrs.pickle')
                
        # Load self.model            
        with open(model_path, 'rb') as file:
            self.model = pickle.load(file)

        # Load attrs            
        with open(attrs_path, 'rb') as file:
            attrs = pickle.load(file)
            
        for attr_name, attr_value in attrs.items():
            setattr(self, attr_name, attr_value)                

### Hyperparameter Tunning & Model Selection

In [132]:
train_periods = int(y.shape[0] * 0.8)
test_periods = int(y.shape[0] * 0.2)

X_train, X_test = X_stand.iloc[:train_periods], X_stand.iloc[train_periods:train_periods+test_periods]
y_train, y_test = y.iloc[:train_periods], y.iloc[train_periods:train_periods+test_periods]
X_forecast = X_stand.iloc[train_periods+test_periods:]

In [ ]:
X_train.shape, X_test.shape

In [ ]:
y_train.shape, y_test.shape

In [135]:
# Define the search space
algorithms = [
    # 'expo_smooth', 
    # 'sarimax',     
    'random_forest',
    'lightgbm',
    # 'lstm',
]

int_parameters = [
    # expo_smooth
    'seasonal_periods',
    
    # sarimax
    'p', 'd', 'q',    
    'seasonal_P',
    'seasonal_D',
    'seasonal_Q',
    'seasonal_S',
    
    # random_forest
    'random_forest.n_estimators',
    'random_forest.max_depth',
    'random_forest.min_samples_split',
    
    # lightgbm
    'lightgbm.n_estimators',
    'lightgbm.max_depth',
    'lightgbm.min_child_samples',
    'num_leaves',
    
    # lstm
    'layers',
    'units',
    'batch_size',
]

choice_parameters = {
    # expo_smooth
    "expo_smooth.trend": ['additive', 'multiplicative', None],
    "damped_trend": [True, False],
    "seasonal": ['additive', 'multiplicative', None],
    "seasonal_periods": [12, 24, 24*7, 24*30],
    
    # sarimax
    "sarimax.trend": [None, 'ct'],
    "seasonal_S": [12, 24], # , 24*7, 24*30],
    
    # random_forest
    "random_forest.max_features": [1.0, 'sqrt'],
    
    # lightgbm
    "boosting_type": ['gbdt', 'dart'],
    
    # lstm
    "topology": ['classic', 'bidirectional', 'convolutional'],
    "units": [16, 32, 64, 128, 256],
    "batch_size": [16, 32, 64, 128, 256]
}

model_type_choices = [
    # Exponential Smoothing Search Space
#     {
#         "algorithm": 'expo_smooth',
#         "expo_smooth.trend": hp.choice('expo_smooth.trend', choice_parameters['expo_smooth.trend']),
#         "damped_trend": hp.choice('damped_trend', choice_parameters['damped_trend']),
#         "seasonal": hp.choice('seasonal', choice_parameters['seasonal']),
#         "seasonal_periods": scope.int(hp.choice('seasonal_periods', choice_parameters['seasonal_periods']))
#     },
    
    # SARIMAX Search Space
#     {
#         "algorithm": 'sarimax',
#         "sarimax.trend": hp.choice('sarimax.trend', choice_parameters['sarimax.trend']),
#         "p": scope.int(hp.quniform('p', 0, 6, 1)),
#         # "d": scope.int(hp.quniform('d', 0, 1, 1)),
#         "q": scope.int(hp.quniform('q', 0, 3, 1)),
#         "seasonal_P": scope.int(hp.quniform('seasonal_P', 0, 3, 1)),
#         # "seasonal_D": scope.int(hp.quniform('seasonal_D', 0, 1, 1)),
#         "seasonal_Q": scope.int(hp.quniform('seasonal_Q', 0, 2, 1)),
#         "seasonal_S": scope.int(hp.choice('seasonal_S', choice_parameters['seasonal_S']))
#     },
    
    # Random Forest Search Space
    {
        "algorithm": 'random_forest',
        "random_forest.n_estimators": scope.int(hp.quniform('random_forest.n_estimators', 5, 200, 1)),
        "random_forest.max_depth": scope.int(hp.quniform('random_forest.max_depth', 1, 100, 1)),
        "random_forest.min_samples_split": scope.int(hp.quniform('random_forest.min_samples_split', 5, 100, 1)),
        "random_forest.max_features": hp.choice('random_forest.max_features', choice_parameters['random_forest.max_features']),
    },
    
    # LightGBM Search Space
    {
        "algorithm": 'lightgbm',
        "boosting_type": hp.choice('boosting_type', choice_parameters['boosting_type']),
        "lightgbm.n_estimators": scope.int(hp.quniform('lightgbm.n_estimators', 5, 350, 1)),
        "lightgbm.max_depth": scope.int(hp.quniform('lightgbm.max_depth', 1, 175, 1)),
        "lightgbm.min_child_samples": scope.int(hp.quniform('lightgbm.min_child_samples', 5, 100, 1)),
        "lightgbm.learning_rate": hp.loguniform('lightgbm.learning_rate', np.log(0.001), np.log(0.3)),
        "num_leaves": scope.int(hp.quniform('num_leaves', 5, 150, 1)),        
        "colsample_bytree": hp.uniform('colsample_bytree', 0.5, 1)
    },
    
    # LSTM Search Space
    # {
    #     "algorithm": 'lstm',
    #     # "topology": hp.choice('topology', choice_parameters['topology']),
    #     "layers": scope.int(hp.quniform('layers', 1, 10, 1)), 
    #     "units": scope.int(hp.choice('units', choice_parameters['units'])),
    #     "dropout": hp.uniform('dropout', 0.0, 0.5),
    #     "recurrent_dropout": hp.uniform('recurrent_dropout', 0.0, 0.5),
    #     "lstm.learning_rate": hp.loguniform('lstm.learning_rate', np.log(0.001), np.log(0.1)),
    #     # "batch_size": scope.int(hp.choice('batch_size', choice_parameters['batch_size'])),
    # },
]

search_space = {
    "model_type": hp.choice('model_type', model_type_choices)
}

In [136]:
def fix_params(params: dict):
    # SARIMAX
    if params['model_type']['algorithm'] == 'sarimax':
        if 'd' not in params['model_type']:
            params['model_type']['d'] = 0
            
        if 'seasonal_D' not in params['model_type']:
            params['model_type']['seasonal_D'] = 0
            
        if params['model_type']['seasonal_P'] == params['model_type']['seasonal_Q'] == 0:
            params['model_type']['seasonal_S'] = 0
            
        if params['model_type']['q'] == params['model_type']['seasonal_Q']:
            if params['model_type']['q'] > 0:
                params['model_type']['q'] += 1
                
        if params['model_type']['p'] == params['model_type']['seasonal_P']:
            params['model_type']['p'] += 1
    
    # Exponential Smoothing
    if params['model_type']['algorithm'] == 'expo_smooth':
        if params['model_type']['expo_smooth.trend'] is None:
            params['model_type']['damped_trend'] = False
            
    return params


def find_trials():
    def extract_param_idx(name, value):
        if name in choice_parameters:
            return choice_parameters[name].index(value)
        return value
    
    def find_trials_dict(champion: bool = True):        
        # Load Champion
        model = Model()
        model.load(champion=champion)

        # Define dict
        trials_dict = {
            'model_type': algorithms.index(model.algorithm)
        }
        trials_dict.update({
            k: extract_param_idx(k, v) for k, v in model.hyper_parameters.items()
        })
        
        return trials_dict
    
    trials_list = [find_trials_dict(champ) for champ in [True, False]]
    
    return generate_trials_to_calculate(trials_list)


def objective(params: dict):
    trials.append(params)
    # try:
    # Extract algorithm
    algorithm = params['model_type']['algorithm']

    # Fix params
    params = fix_params(params)

    # Extract Hyper Parameters
    hyper_parameters = {
        k: v for k, v in params['model_type'].items() if k != 'algorithm'
    }

    ts_split = TimeSeriesSplit(n_splits=3, test_size=test_periods)

    scores = []

    for train_idx, val_idx in ts_split.split(X_train):
        # Split Features
        train_features = X_train.iloc[train_idx]
        val_features = X_train.iloc[val_idx]

        # Split Target
        train_target = y_train.iloc[train_idx]
        val_target = y_train.iloc[val_idx]

        # Instanciate the Model
        model = Model(
            algorithm=algorithm,
            hyper_parameters=hyper_parameters,
        )

        # Build the model
        model.build(
            train_target=train_target.copy(), 
            train_features=train_features.copy()
        )

        # Fit the model
        model.fit(
            train_target=train_target.copy(), 
            train_features=train_features.copy()
        )

        # Predict the Validation Dataset
        val_pred = model.predict(
            train_target=train_target.copy(), 
            forecast_features=val_features.copy(), 
            forecast_dates=val_features.index
        )

        scores.append(mean_absolute_percentage_error(val_target, val_pred))

    return {'loss': np.mean(scores), 'status': STATUS_OK}
#     except Exception as e:
#         logger.warning(f"Unable to run {params}.\n"
#                        f"Exception {e}")
#         return {'loss': np.inf, 'status': STATUS_OK}

In [137]:
# find_trials()

In [ ]:
trials = []

result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    # trials=find_trials(),
    max_evals=300,
    timeout=10 * 60, # 30 mins
    verbose=True,
    show_progressbar=True,
    early_stop_fn=None
)

In [ ]:
result

In [ ]:
algos = [t['model_type']['algorithm'] for t in trials]
counts = Counter(algos)

for algo, reps in counts.items():
    print(f"{algo}: {round(reps * 100 / len(algos), 1)} %")

### Model Evaluation

In [141]:
# Instanciate the "challenger" model

In [142]:
def extract_challenger(results: dict):
    def extract_parameter(k, v):
        if k in choice_parameters:
            v = choice_parameters[k][v]
        if k in int_parameters:
            v = int(v)
        return v
    
    algorithm = algorithms[results['model_type']]
    
    hyper_parameters = {
        k: extract_parameter(k, v) for k, v in results.items() if k not in ['algorithm', 'model_type']
    }
    
    return Model(
        algorithm=algorithm,
        hyper_parameters=hyper_parameters
    )

In [143]:
challenger = extract_challenger(result)

In [ ]:
challenger

In [145]:
# Build & Re-fit model
challenger.build(
    train_target=y_train, 
    train_features=X_train
)

challenger.fit(
    train_target=y_train, 
    train_features=X_train
)

In [ ]:
# Save Challenger
challenger.save(as_champion=False)

In [147]:
# Forecast Test Data
test_pred = challenger.predict(
    train_target=y_train, 
    forecast_features=X_test, 
    forecast_dates=X_test.index
)

In [148]:
# Evaluate Performance (on unseen data)

In [149]:
challenger_mape = mean_absolute_percentage_error(y_test, test_pred)

In [ ]:
challenger_mape

### Champion Update

In [ ]:
if os.path.exists(os.path.join('models', 'champion', 'champion.pickle')):
    # Load Champion
    champion = Model()
    champion.load(champion=True)
    
    # Fit champion
    champion.fit(
        train_target=y_train, 
        train_features=X_train
    )
    
    # Forecast test data
    test_pred = champion.predict(
        train_target=y_train, 
        forecast_features=X_test, 
        forecast_dates=X_test.index
    )
    
    # Evaluate champion
    champion_mape = mean_absolute_percentage_error(y_test, test_pred)
    
    print(f'champion_mape: {round(champion_mape*100, 2)} %\n'
          f'challenger_mape: {round(challenger_mape*100, 2)} %\n')
    
    if challenger_mape < champion_mape:
        challenger.save(as_champion=True)
else:
    challenger.save(as_champion=True)